Exploration of what a possible pipeline could look like for processing a URL from start to finish.

Once finalized this can be implemented as a series of methods in `pipeline.py`.

Some questions to discuss:
    
    - Should we attempt end-to-end processing in one shot or split it into steps
    - Implemente these as class methods, then initialize Pipeline() and carry out the steps one at a time?

This also highlights some pending issues to deal with in the Scraper and/or Interpreter code, i.e.

1. Turn extracted dates into Date Windows
2. Turn extracted quantities into integers for saving to DB
3. Add step for classifying article

#### Set up the db

In [1]:
%load_ext autoreload
%autoreload 2
# Change these to True to set up the DB the first time
i_know_this_will_delete_everything = True
initialize_id_test = True
initialize_id = False

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from internal_displacement.model.model import init_db

db_host = os.environ.get('DB_HOST')

if initialize_id:
    db_url = 'postgresql://{user}:{password}@{db_host}/{db}'.format(
        user='jupyter', password='jupyter', db_host=db_host, db='id')
    init_db(db_url, i_know_this_will_delete_everything=i_know_this_will_delete_everything)
    
if initialize_id_test:
    db_url = 'postgresql://{user}:{password}@{db_host}/{db}'.format(
        user='tester', password='tester', db_host=db_host, db='id_test')
    init_db(db_url, i_know_this_will_delete_everything=i_know_this_will_delete_everything)

In [2]:
import spacy
import json
from sqlalchemy import create_engine
from datetime import datetime
from internal_displacement.model.model import Status, Session, Category, Article, Content, Country, CountryTerm, \
    Location, Report, ReportDateSpan, ArticleCategory, Base
from internal_displacement.scraper import Scraper
from internal_displacement.interpreter import Interpreter
from internal_displacement.pipeline import Pipeline

#### Set-up the session

In [3]:
engine = create_engine(db_url)
Session.configure(bind=engine)
session = Session()

#### Initialize scraper and interpreter

In [4]:
scraper = Scraper()
nlp = spacy.load('en')
person_reporting_terms = [
    'displaced', 'evacuated', 'forced', 'flee', 'homeless', 'relief camp',
    'sheltered', 'relocated', 'stranded', 'stuck', 'stranded', "killed", "dead", "died", "drown"
]

structure_reporting_terms = [
    'destroyed', 'damaged', 'swept', 'collapsed',
    'flooded', 'washed', 'inundated', 'evacuate'
]

person_reporting_units = ["families", "person", "people", "individuals", "locals", "villagers", "residents",
                            "occupants", "citizens", "households", "life"]

structure_reporting_units = ["home", "house", "hut", "dwelling", "building", "shop", "business", "apartment",
                                     "flat", "residence"]

relevant_article_terms = ['Rainstorm', 'hurricane',
                          'tornado', 'rain', 'storm', 'earthquake']
relevant_article_lemmas = [t.lemma_ for t in nlp(
    " ".join(relevant_article_terms))]

data_path = '../data'
interpreter = Interpreter(nlp, person_reporting_terms, structure_reporting_terms, person_reporting_units,
                          structure_reporting_units, relevant_article_lemmas, data_path)

#### Initialize Pipeline

In [5]:
pipeline = Pipeline(session, scraper, interpreter)

#### Invalid url

In [6]:
url = 'http://www.independent.co.uk/news/somefakenewsstory'
pipeline.process_url(url)

'fetching failed'

#### Non-english url

In [7]:
url = 'http://www.eluniversal.com.mx/articulo/nacion/politica/2017/03/13/manifestantes-obligan-acortar-evento-de-amlo-en-ny'
pipeline.process_url(url)

'Processed: Not in English'

#### Irrelevant url

In [8]:
url = 'http://www.bbc.com/news/world-europe-39258436'
pipeline.process_url(url)

'Processed: Not relevant'

#### Relevant url

In [9]:
url = 'http://www.independent.co.uk/news/world/asia/160-killed-and-hundreds-left-stranded-by-flooding-across-afghanistan-and-pakistan-8746566.html'
pipeline.process_url(url)

'processed'

#### Clean-up Database

In [34]:
test_urls = [
    'http://www.independent.co.uk/news/somefakenewsstory',
    'http://www.eluniversal.com.mx/articulo/nacion/politica/2017/03/13/manifestantes-obligan-acortar-evento-de-amlo-en-ny',
    'http://www.bbc.com/news/world-europe-39258436',
    'http://www.independent.co.uk/news/world/asia/160-killed-and-hundreds-left-stranded-by-flooding-across-afghanistan-and-pakistan-8746566.html'
]

In [35]:
for url in test_urls:
    session.query(Article).filter_by(url=url).delete()

In [36]:
session.commit()